# CIFAR10 Image Classifier Explanations


We will use a Tensorflow classifier built on [CIFAR10 image dataset](https://www.cs.toronto.edu/~kriz/cifar.html) which is a 10 class image dataset.

The Kfserving resource describes:
   * A pretrained tensorflow model stored on a Google bucket
   * Am AnchorImage [Seldon Alibi](https://github.com/SeldonIO/alibi) Explainer. See the [Alibi Docs](https://docs.seldon.io/projects/alibi/en/stable/) for further details.
   
To download and get the pretrained image classifier locally you can run the following code:

```python
url = "https://storage.googleapis.com/seldon-models/alibi-detect/classifier/"
path_model = os.path.join(url, "cifar10", "resnet32", "model.h5")
save_path = tf.keras.utils.get_file("resnet32", path_model)
model = tf.keras.models.load_model(save_path)
```

In [ ]:
!pygmentize cifar10.yaml

In [ ]:
!kubectl apply -f cifar10.yaml -n default

In [ ]:
!kubectl wait --for condition=ready --timeout=600s pods --all -n default

In [ ]:
CLUSTER_IPS=!(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
CLUSTER_IP=CLUSTER_IPS[0]
print(CLUSTER_IP)

In [ ]:
SERVICE_HOSTNAMES=!(kubectl get inferenceservice cifar10 -n default -o jsonpath='{.status.url}' | cut -d "/" -f 3)
SERVICE_HOSTNAME=SERVICE_HOSTNAMES[0]
print(SERVICE_HOSTNAME)

In [ ]:
import sys
sys.path.append('../')
from alibi_helper import *

In [ ]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf

train, test = tf.keras.datasets.cifar10.load_data()
X_train, y_train = train
X_test, y_test = test

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [ ]:
import json
from subprocess import PIPE, Popen, run

import numpy as np

idx = 12
test_example = X_test[idx : idx + 1].tolist()
payload = '{"instances":' + f"{test_example}" + " }"
cmd = f"""curl -s -d '{payload}' \
   http://{CLUSTER_IP}/v1/models/cifar10:predict \
   -H "Host: {SERVICE_HOSTNAME}" \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True, stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
res = json.loads(raw)
print(res)
arr = np.array(res["predictions"])
X = X_test[idx].reshape(1, 32, 32, 3)
plt.imshow(X.reshape(32, 32, 3))
plt.axis("off")
plt.show()
print("class:", class_names[y_test[idx][0]])
print("prediction:", class_names[arr[0].argmax()])

In [ ]:
test_example = X_test[idx : idx + 1].tolist()
payload = '{"instances":' + f"{test_example}" + " }"
cmd = f"""curl -s -d '{payload}' \
   http://{CLUSTER_IP}/v1/models/cifar10:explain \
   -H "Host: {SERVICE_HOSTNAME}" \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True, stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
explanation = json.loads(raw)
arr = np.array(explanation["data"]["anchor"])
plt.imshow(arr)

## Teardown

In [ ]:
!kubectl delete -f cifar10.yaml -n default